In [1]:
from general_tools.notebook.gpu_utils import setup_one_gpu
GPU = 2
setup_one_gpu(GPU)

Picking GPU 2


In [2]:
import sys
import time
import numpy as np
import os.path as osp
import tensorflow as tf
import matplotlib.pyplot as plt

from general_tools.notebook.tf import reset_tf_graph
from general_tools.in_out import create_dir
from general_tools.in_out.basics import create_dir, delete_files_in_directory, files_in_subdirs

from geo_tool import Point_Cloud

from tf_lab.in_out.basics import Data_Splitter, read_saved_epochs
from tf_lab.point_clouds.ae_templates import conv_architecture_ala_nips_17, default_train_params_ala_nips_17
from tf_lab.point_clouds.encoders_decoders import encoder_with_convs_and_symmetry, decoder_with_fc_only
from tf_lab.point_clouds.point_net_ae import PointNetAutoEncoder
from tf_lab.point_clouds.autoencoder import Configuration as Conf
from tf_lab.point_clouds.in_out import load_point_clouds_from_filenames, PointCloudDataSet
from tf_lab.data_sets.shape_net import pc_loader as snc_loader
from tf_lab.data_sets.shape_net import snc_category_to_synth_id

from pc_completions.evaluation import basic_comletion_measures

In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [5]:
n_pc_samples = 2048
do_training = True
first_time_running = True
load_model = False
seed = 42
max_training_epochs = 2000
loss = 'chamfer'
z_rotate = True
experiment_name = '2_all_snc_rotated_conv_arch_with_bnorm_' + str(n_pc_samples) +  'pts_' + loss

In [6]:
top_data_dir = '/orions4-zfs/projects/optas/DATA/'
full_pclouds_path = osp.join(top_data_dir, 'Point_Clouds/Shape_Net/Core/from_manifold_meshes/centered/', str(n_pc_samples))
train_dir = osp.join(top_data_dir, 'OUT/iclr/nn_models/all_snc', experiment_name)
create_dir(train_dir)

'/orions4-zfs/projects/optas/DATA/OUT/iclr/nn_models/all_snc/2_all_snc_rotated_conv_arch_with_bnorm_2048pts_chamfer'

In [7]:
n_threads = 40
verbose = True
full_file_names = [f for f in files_in_subdirs(full_pclouds_path, '.ply')]
pclouds, model_names, syn_ids = load_point_clouds_from_filenames(full_file_names, n_threads, snc_loader, verbose)
model_unames = model_names + '.' + syn_ids
train_data = PointCloudDataSet(pclouds, labels=model_unames)

/orions4-zfs/projects/optas/Git_Repos/tf_lab/point_clouds/in_out.py:38: UserWarning: Point clouds with the same model name were loaded.
  warnings.warn('Point clouds with the same model name were loaded.')


57449 pclouds were loaded. They belong in 57 shape-classes.


In [8]:
n_input = [n_pc_samples, 3]

encoder_args = {'n_filters': [128, 128, 256, 512, 512],
                'filter_sizes': [16, 2, 2, 2, 2],
                'strides': [1, 1, 1, 1, 1],
                'b_norm': True,                
               }

decoder_args = {'layer_sizes': [512, 1024, np.prod(n_input)],
                'b_norm': True
               }

if load_model:
    conf = Conf.load(osp.join(train_dir, 'configuration'))
    print conf
    
else:    
    conf = Conf(
                n_input = n_input,
                training_epochs = max_training_epochs,
                batch_size = 50,
                loss = loss,
                denoising = False,
                train_dir = train_dir,
                loss_display_step = 1,
                saver_step = 10,
                learning_rate = 0.0005,
                z_rotate = z_rotate,
                encoder = encoder_with_convs_and_symmetry,
                encoder_args = encoder_args,
                decoder = decoder_with_fc_only,
                decoder_args = decoder_args,            
               )
    conf.allow_gpu_growth = False
    conf.experiment_name = experiment_name
    conf.save(osp.join(conf.train_dir, 'configuration'))

In [ ]:
reset_tf_graph()
ae = PointNetAutoEncoder(experiment_name, conf)

if load_model:
    saved_epochs = read_saved_epochs(conf.train_dir)
    last_epoch = saved_epochs[-1]
    ae.restore_model(conf.train_dir, last_epoch, verbose=True)

In [ ]:
if do_training:
    buf_size = 1 # flush each line
    fout = open(osp.join(conf.train_dir, 'train_stats.txt'), 'a', buf_size)    
    train_stats = ae.train(train_data, conf, log_file=fout)
    fout.close()

('Epoch:', '0001', 'training time (minutes)=', '3.8081', 'loss=', '0.008803837')
INFO:tensorflow:/orions4-zfs/projects/optas/DATA/OUT/iclr/nn_models/all_snc/2_all_snc_rotated_conv_arch_with_bnorm_2048pts_chamfer/models.ckpt-1 is not in all_model_checkpoint_paths. Manually adding it.
('Epoch:', '0002', 'training time (minutes)=', '3.8332', 'loss=', '0.003423860')
('Epoch:', '0003', 'training time (minutes)=', '3.8421', 'loss=', '0.002393697')
('Epoch:', '0004', 'training time (minutes)=', '3.8520', 'loss=', '0.001891442')
('Epoch:', '0005', 'training time (minutes)=', '3.8505', 'loss=', '0.001796006')
('Epoch:', '0006', 'training time (minutes)=', '3.8521', 'loss=', '0.001855912')
('Epoch:', '0007', 'training time (minutes)=', '3.8438', 'loss=', '0.001632064')
('Epoch:', '0008', 'training time (minutes)=', '3.8460', 'loss=', '0.001442132')
('Epoch:', '0009', 'training time (minutes)=', '3.8468', 'loss=', '0.001338968')
('Epoch:', '0010', 'training time (minutes)=', '3.8491', 'loss=', '0